Rozwiązywanie układów równań liniowych metodą iteracyjną Jacobiego

Wykorzystane moduły

In [2]:
import numpy as np
from enum import Enum
from functools import wraps
import time

![title](img_vsc/img01.png)

In [3]:
# Kryteria przyrostowe
class Condition(Enum):
    
    FIRST = lambda X_approx,  X, eps: np.linalg.norm(X_approx - X) < eps 
    SECOND = lambda A, X, b, eps: np.linalg.norm(A @ X - b) < eps

In [4]:
def measure_jacobi_time(func):
    
    @wraps(func)
    def wrapper(*args):
        
        start = time.perf_counter()
        
        X, it = func(*args)
        
        end = time.perf_counter()

        return X, it, end - start
    return wrapper

In [130]:
class JacobiTest():
    
    k = 8
    m = 2
    
    @measure_jacobi_time
    @staticmethod
    def jacobi(A, b, condition, eps, max_iterations):
        
        D = np.diag(A).astype(np.float64) # Przekątna macierzy A
        C = A - np.diagflat(D).astype(np.float64) # A bez przekątnej
        X = np.zeros_like(b).astype(np.float64) # Tablica zerowa o takim samym kształcie jak b
        
        for it in range(max_iterations):
            X_approx = (b - (C @ X)) / D
            
            if condition == Condition.FIRST and Condition.FIRST(X_approx, X, eps):
                break
            elif condition == Condition.SECOND and Condition.SECOND(A, X, b, eps):
                break
            X = X_approx
            
        return X, it
    
    @classmethod
    def __get_matrix(cls, n):
        return np.array([[cls.k if i == j else 1 / (abs(i - j) + cls.m) for j in range(n)] for i in range(n)], dtype=np.float64)
    
    @staticmethod
    def __get_vector(n):
        return np.array([1 if i % 2 == 0 else -1 for i in range(n)], dtype=np.float64)
        
    @classmethod
    def get_system(cls, n):
        
        A = cls.__get_matrix(n)
        X = cls.__get_vector(n)
        
        return A, A @ X, X
    
    @classmethod
    def jacobi_n(cls, systems, eps, condition, max_iterations = 1000):
        
        result = []
        
        for A, b, X in systems:
                        
            X_approx, iterations, duration = cls.jacobi(A, b, condition, eps, max_iterations)
                    
            result.append((np.linalg.norm(X - X_approx), iterations, duration))
            
        return result      

In [120]:
n = [3, 5, 7, 10, 15, 20, 35, 50, 75, 100, 125, 150, 200, 250, 300, 350]
eps = [1e-3, 1e-7, 1e-15, 1e-20]
systems = list(map(lambda x : JacobiTest().get_system(x), n))

Precyzja 1e-3

In [8]:
results_first_1e_3, results_second_1e_3 = JacobiTest.jacobi_n(systems, 1e-3, Condition.FIRST, 1000), JacobiTest.jacobi_n(systems, 1e-3, Condition.SECOND)

In [9]:
for i, (error, iterations, duration) in enumerate(results_first_1e_3):
    print(n[i], error, iterations, duration)

3 0.00027657023415481746 3 0.0001862000003711728
5 0.00011797844999045876 4 7.400000004054164e-05
7 0.00030706141805554826 4 7.29000003047986e-05
10 0.0003370163472400488 3 6.619999976464896e-05
15 0.00047669834508336456 5 0.00019500000007610652
20 0.000651468788308314 3 7.060000007186318e-05
35 0.0005579595453482399 7 8.650000017951243e-05
50 0.0006949789352982175 4 9.36000001274806e-05
75 0.000493832536680288 11 0.0001467000001866836
100 0.00039873796638740367 6 0.0020349999999780266
125 0.0004500863757870157 17 0.0025012000000970147
150 0.000494924079886817 7 0.0013346000000638014
200 0.0005521724962843907 8 0.001660399999764195
250 0.0005999514339342845 9 0.0018657999999049935
300 0.000439484401750828 11 0.0029385999996520695
350 0.0005070354198577723 12 0.0036531000000650238


In [10]:
for i, (error, iterations, duration) in enumerate(results_second_1e_3):
    print(n[i], error, iterations, duration)

3 1.892207222489987e-05 4 9.390000013809185e-05
5 1.5671221205628294e-05 5 7.639999967068434e-05
7 5.4797787725071414e-05 5 6.930000017746352e-05
10 1.6083207913400562e-05 4 5.939999982729205e-05
15 4.3848971999910014e-05 7 8.520000028511276e-05
20 6.0163907038124154e-05 4 5.6599999879836105e-05
35 5.8395157059540475e-05 10 0.00010900000006586197
50 5.561585132904874e-05 6 8.099999968180782e-05
75 5.3117864274722075e-05 16 0.0002448000000185857
100 7.196233810393804e-05 8 0.0030876000000716886
125 6.556866221638224e-05 24 0.007539300000189542
150 6.737289647735507e-05 10 0.0040911000000960485
200 6.256286635347576e-05 12 0.003014999999777501
250 6.056900370161495e-05 14 0.003384299999652285
300 6.166575360887368e-05 16 0.004887000000053376
350 6.595897929457682e-05 18 0.005406399999628775


Precyzja 1e-7

In [11]:
results_first_1e_7, results_second_1e_7 = JacobiTest.jacobi_n(systems, 1e-7, Condition.FIRST, 1000), JacobiTest.jacobi_n(systems, 1e-7, Condition.SECOND)

In [12]:
for i, (error, iterations, duration) in enumerate(results_first_1e_7):
    print(n[i], error, iterations, duration)

3 7.925528275653803e-09 7 0.00017719999959808774
5 3.708892142531895e-08 8 8.519999983036541e-05
7 5.576016066291517e-08 9 7.790000017848797e-05
10 3.673605066208856e-08 6 6.00999997004692e-05
15 3.4127886906484605e-08 13 0.0002548000002207118
20 1.5229267214950545e-08 8 0.00017300000035902485
35 6.694208212390076e-08 19 0.0003774000001612876
50 2.8773489493222252e-08 12 0.0001432999997632578
75 4.232095926329985e-08 32 0.00037920000022495515
100 3.2589341528906207e-08 17 0.00379190000012386
125 5.120667635624557e-08 50 0.008143000000018219
150 4.518212555072474e-08 21 0.0040911000000960485
200 5.2989810336152925e-08 25 0.00526960000024701
250 3.951210815641748e-08 30 0.0058779999999387655
300 5.24963999093285e-08 34 0.00489880000031917
350 5.2419108850599144e-08 39 0.0059144000001651875


In [13]:
for i, (error, iterations, duration) in enumerate(results_second_1e_7):
    print(n[i], error, iterations, duration)

3 7.925528275653803e-09 7 0.00011319999975967221
5 4.943431302229421e-09 9 0.0001007999999274034
7 9.959123360247537e-09 10 0.00011230000018258579
10 1.7574842295615031e-09 7 8.210000032704556e-05
15 3.139251112367329e-09 15 0.00014280000004873727
20 1.9610433428683717e-09 9 0.00010519999977987027
35 7.006051855814652e-09 22 0.00021440000000438886
50 8.152635404040681e-09 13 0.0001540000002933084
75 7.110239489259351e-09 36 0.0004034999997202249
100 5.888064149850948e-09 19 0.004083799999989424
125 5.665168605223369e-09 58 0.011962799999764684
150 6.158472188026357e-09 24 0.005882699999801844
200 6.0107683411578005e-09 29 0.006519700000353623
250 6.314700378720415e-09 34 0.007538299999851006
300 7.368462017964859e-09 39 0.009354400000120222
350 6.820836237137304e-09 45 0.011234600000079809


Precyzja 1e-15

In [14]:
results_first_1e_15, results_second_1e_15 = JacobiTest.jacobi_n(systems, 1e-15, Condition.FIRST), JacobiTest.jacobi_n(systems, 1e-15, Condition.SECOND)

In [15]:
for i, (error, iterations, duration) in enumerate(results_first_1e_15):
    print(n[i], error, iterations, duration)

3 2.220446049250313e-16 14 0.0002134999999725551
5 5.551115123125783e-16 17 0.00029140000015104306
7 4.1540741810552243e-16 20 0.00027139999974679085
10 6.080941944488118e-16 12 0.0001063999998223153
15 7.771561172376096e-16 28 0.0007379999997283448
20 5.978733960281817e-16 17 0.00016459999960716232
35 1.3642648929939613e-15 44 0.00031499999977313564
50 1.7728839976828627e-15 27 0.0002129999998032872
75 1.9860273225978185e-15 72 0.0005802000000585394
100 1.877557501171774e-15 38 0.00760809999974299
125 2.3733857464271978e-15 113 0.017221000000063214
150 3.7105374255538066e-15 50 0.006255000000237487
200 5.074338674593921e-15 62 0.008298999999624357
250 6.654858695507619e-15 80 0.00926709999976083
300 8.015933253251316e-15 94 0.012456799999654322
350 9.222205069512406e-15 999 0.11509949999981473


In [16]:
for i, (error, iterations, duration) in enumerate(results_second_1e_15):
    print(n[i], error, iterations, duration)

3 0.0 15 0.00017389999993611127
5 1.5700924586837752e-16 18 0.0001677999998719315
7 1.1102230246251565e-16 22 0.0001870999999482592
10 2.7194799110210365e-16 13 0.00011690000019370927
15 5.324442579404919e-16 999 0.008665800000017043
20 6.568167990716596e-16 999 0.007611999999880936
35 1.2412670766236366e-15 999 0.008279599999696075
50 1.7199501139797033e-15 999 0.008487300000069808
75 1.870981097424087e-15 999 0.010976900000059686
100 1.7832822334594722e-15 999 0.27018550000002506
125 2.336749147496729e-15 999 0.2019669999999678
150 3.66709875019605e-15 999 0.1979833000000326
200 5.056087786996822e-15 999 0.20706569999993008
250 6.650226650194185e-15 999 0.2095766999996158
300 8.015933253251316e-15 999 0.20781129999977566
350 9.222205069512406e-15 999 0.2147132000000056


Precyzja 1e-20 - to nie będzie w sprawozdaniu bo wyniki są niemal identyczne do tych powyżej

In [17]:
results_first_1e_20, results_second_1e_20 = JacobiTest.jacobi_n(systems, 1e-20, Condition.FIRST), JacobiTest.jacobi_n(systems, 1e-20, Condition.SECOND)

In [18]:
for i, (error, iterations, duration) in enumerate(results_first_1e_20):
    print(n[i], error, iterations, duration)

3 0.0 15 0.00033660000008239876
5 0.0 20 0.00016520000008313218
7 1.1102230246251565e-16 22 0.0001654000002417888
10 2.482534153247273e-16 14 0.00010740000016085105
15 5.324442579404919e-16 30 0.00021140000035302364
20 6.568167990716596e-16 19 0.00013699999999516876
35 1.2412670766236366e-15 50 0.00035419999994701357
50 1.7199501139797033e-15 30 0.0002351999996790255
75 1.870981097424087e-15 75 0.0006253999999898952
100 1.7832822334594722e-15 999 0.113822600000276
125 2.336749147496729e-15 123 0.014308899999832647
150 3.66709875019605e-15 999 0.1133641000001262
200 5.056087786996822e-15 69 0.008174099999905593
250 6.650226650194185e-15 999 0.11528659999976298
300 8.015933253251316e-15 999 0.10799230000020543
350 9.222205069512406e-15 999 0.11438369999996212


In [18]:
for i, (error, iterations, duration) in enumerate(results_second_1e_20):
    print(n[i], error, iterations, duration)

3 0.0 15 0.00017400000001543958
5 0.0 20 0.0001799000001483364
7 1.1102230246251565e-16 999 0.00725430000011329
10 2.7194799110210365e-16 13 0.00012700000002041634
15 5.324442579404919e-16 999 0.011057700000037585
20 6.568167990716596e-16 999 0.013115400000060617
35 1.2412670766236366e-15 999 0.015299299999924187
50 1.7199501139797033e-15 999 0.009631099999978687
75 1.870981097424087e-15 999 0.010780700000168508
100 1.7832822334594722e-15 999 0.20967359999986002
125 2.336749147496729e-15 999 0.2055214999998043
150 3.66709875019605e-15 999 0.2246895000000677
200 5.056087786996822e-15 999 0.22655239999994592
250 6.650226650194185e-15 999 0.2143564000000424
300 8.015933253251316e-15 999 0.2179602999999588
350 9.222205069512406e-15 999 0.2300930999999764


Wyznaczanie promienia spektralnego

In [127]:
def spectral_radius(A):
    B = np.diagflat(np.diag(A)).astype(np.float64)
    I = np.eye(A.shape[0]).astype(np.float64)
    M = I - np.linalg.inv(B) @ A
    return max(abs(np.linalg.eigvals(M)))

In [132]:
for i, x in enumerate(map(spectral_radius, map(lambda x : x[0], [JacobiTest().get_system(n) for n in range(1, 500 + 1)]))):
    print("n = {0}, p = {1}".format(i + 1, x))

n = 1, p = 0.0
n = 2, p = 0.041666666666666664
n = 3, p = 0.0765869787016647
n = 4, p = 0.106724645640869
n = 5, p = 0.13328604069582112
n = 6, p = 0.15706336209739008
n = 7, p = 0.17860643375742666
n = 8, p = 0.1983135558449416
n = 9, p = 0.21648316701401726
n = 10, p = 0.2333450543295334
n = 11, p = 0.24908015871441297
n = 12, p = 0.26383366218814525
n = 13, p = 0.2777239327893888
n = 14, p = 0.2908488156028606
n = 15, p = 0.3032901670949366
n = 16, p = 0.3151171935109013
n = 17, p = 0.3263889549414487
n = 18, p = 0.33715627470070875
n = 19, p = 0.3474632166951742
n = 20, p = 0.35734824359906625
n = 21, p = 0.3668451355778974
n = 22, p = 0.37598372689889514
n = 23, p = 0.3847905023018279
n = 24, p = 0.39328908414344915
n = 25, p = 0.40150063358230315
n = 26, p = 0.4094441834658544
n = 27, p = 0.4171369164733274
n = 28, p = 0.4245943990193639
n = 29, p = 0.43183077913667955
n = 30, p = 0.4388589548223364
n = 31, p = 0.4456907180055828
n = 32, p = 0.4523368782706767
n = 33, p = 0.45880

In [133]:
for x in map(spectral_radius, map(lambda x : x[0], [JacobiTest().get_system(n) for n in range(1, 500 + 1)])):
    print(x)

0.0
0.041666666666666664
0.0765869787016647
0.106724645640869
0.13328604069582112
0.15706336209739008
0.17860643375742666
0.1983135558449416
0.21648316701401726
0.2333450543295334
0.24908015871441297
0.26383366218814525
0.2777239327893888
0.2908488156028606
0.3032901670949366
0.3151171935109013
0.3263889549414487
0.33715627470070875
0.3474632166951742
0.35734824359906625
0.3668451355778974
0.37598372689889514
0.3847905023018279
0.39328908414344915
0.40150063358230315
0.4094441834658544
0.4171369164733274
0.4245943990193639
0.43183077913667955
0.4388589548223364
0.4456907180055828
0.4523368782706767
0.458807369670419
0.4651113433400802
0.47125724812641634
0.477252901052582
0.483105549123955
0.48882192372551836
0.49440828865499753
0.4998704826677595
0.5052139572715728
0.5104438103957458
0.5155648164651747
0.5205814533317326
0.5254979264501458
0.5303181906309204
0.5350459696567984
0.5396847740104069
0.544237916927837
0.5487085289648452
0.5530995712384612
0.5574138474863866
0.5616540150689